<a href="https://colab.research.google.com/github/Margo-s-M/Portfolio_Python/blob/master/SQL_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth ,files
from google.cloud import bigquery

In [2]:
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

In [3]:
query ="""WITH


accounts_info AS (      #1-cte with comon info about accounts
  SELECT
    s.date AS date,
    sp.country,
    a.send_interval,
    a.is_verified,
    a.is_unsubscribed,
    COUNT(DISTINCT a.id) AS account_cnt,
    0 AS sent_cnt,
    0 AS open_cnt,
    0 AS visit_cnt
  FROM `data-analytics-mate.DA.account` a
  JOIN `data-analytics-mate.DA.account_session` acc_s ON a.id = acc_s.account_id
  JOIN `data-analytics-mate.DA.session` s ON acc_s.ga_session_id = s.ga_session_id
  JOIN `data-analytics-mate.DA.session_params` sp ON s.ga_session_id = sp.ga_session_id
  GROUP BY 1, 2, 3, 4, 5
),




emails_info AS (    #2-cte with emails info
  SELECT
    DATE_ADD(s.date, INTERVAL es.sent_date DAY) AS date,#whith interval only
    sp.country,
    a.send_interval,
    a.is_verified,
    a.is_unsubscribed,
    0 AS account_cnt,        #counting all 3types of mess with dist.
    COUNT(DISTINCT es.id_message) AS sent_cnt,
    COUNT(DISTINCT eo.id_message) AS open_cnt,
    COUNT(DISTINCT ev.id_message) AS visit_cnt
  FROM `data-analytics-mate.DA.email_sent` es
  JOIN `data-analytics-mate.DA.account` a ON es.id_account = a.id
  JOIN `data-analytics-mate.DA.account_session` acc_s ON a.id = acc_s.account_id
  JOIN `data-analytics-mate.DA.session` s ON acc_s.ga_session_id = s.ga_session_id
  JOIN `data-analytics-mate.DA.session_params` sp ON s.ga_session_id = sp.ga_session_id
  LEFT JOIN `data-analytics-mate.DA.email_open` eo ON es.id_message = eo.id_message
  LEFT JOIN `data-analytics-mate.DA.email_visit` ev ON es.id_message = ev.id_message
  GROUP BY 1, 2, 3, 4, 5
),




#1st cte and 2nd conected
union_data AS (
  SELECT * FROM accounts_info
  UNION ALL
  SELECT * FROM emails_info
),




#Aggregation of metrics in the required sections.


total AS (
  SELECT
    date,
    country,
    send_interval,
    is_verified,
    is_unsubscribed,
    SUM(account_cnt) AS account_cnt,
    SUM(sent_cnt) AS sent_msg,
    SUM(open_cnt) AS open_msg,
    SUM(visit_cnt) AS visit_msg
  FROM union_data
  GROUP BY 1, 2, 3, 4, 5
),


#window functions, sum by partittion


with_totals AS (
  SELECT
    *,
    SUM(account_cnt) OVER (PARTITION BY country) AS total_country_account_cnt,
    SUM(sent_msg) OVER (PARTITION BY country) AS total_country_sent_cnt
  FROM total
),


#Ranking by country, using DENSE_RANK
last_cte AS (
  SELECT
    *,
    DENSE_RANK() OVER (ORDER BY total_country_account_cnt DESC) AS rank_total_country_account_cnt,
    DENSE_RANK() OVER (ORDER BY total_country_sent_cnt DESC) AS rank_total_country_sent_cnt
  FROM with_totals
)


# final select country counts top 10
SELECT *
FROM last_cte
WHERE rank_total_country_account_cnt <= 10
   OR rank_total_country_sent_cnt <= 10
ORDER BY date, country;
"""

df = client.query(query).to_dataframe()


df.to_csv('dataset_from_bigquery.csv', index=False)


print(df.head())


         date country  send_interval  is_verified  is_unsubscribed  \
0  2020-11-01  Canada              1            1                0   
1  2020-11-01  Canada              1            1                1   
2  2020-11-01  Canada              1            0                0   
3  2020-11-01   China              1            1                0   
4  2020-11-01  France              1            1                1   

   account_cnt  sent_msg  open_msg  visit_msg  total_country_account_cnt  \
0            8         6         3          0                       2067   
1            3         2         2          0                       2067   
2            5         0         0          0                       2067   
3            2         3         0          0                        490   
4            1         0         0          0                        553   

   total_country_sent_cnt  rank_total_country_account_cnt  \
0                   37914                               3   
